In [1]:
import csv as csv
import json
import urllib
import pandas as pd
import numpy as np
import re
pd.set_option('display.max_rows', 100)

## The data:
1. **Cases from http://supremecourtdatabase.org/data.php**
I used Justice-Centered Data: Cases Organized by Supreme Court Citation
Includes cases from 1946 - 2014.

2. **2015 Cases from https://api.oyez.org/cases/2015**
This includes cases from this current term. Cases are given the year of their fall term.


First I'll read in and ready the 1946 - 2014 database. Then I'll read in and ready the 2015 database and join them together.

### Supreme Court Database
#### Reading in and filtering / grouping the data I need

In [2]:
#Reading in the Supreme Court Database
#Each row reprsents a vote by a justice on a case
raw_votes_46_14 = pd.read_csv('data/2015-03-all-cases.csv', header=0)

In [3]:
#creating a new column for year. 
#Case given the year of the fall half of the term. (So the 2014-2015 term is '2014')
raw_votes_46_14['year'] = raw_votes_46_14['caseId'].apply(lambda a: int(re.search('(.*)-', a).group(1)))

In [4]:
#creating a column to keep track of whether a judge was recused. 
#If the judge is present but has no vote, they were recused or sick but still a member of the court
raw_votes_46_14.loc[raw_votes_46_14['vote'].isnull(), 'recused'] = 1


In [5]:
#selecting columns I need, and turning NaNs in recused column to zeros
filtered_votes_46_14 = raw_votes_46_14[['caseId', 'year', 'majVotes', 'minVotes', 'recused']].fillna(value=0)
filtered_votes_46_14.head()

,caseId,year,majVotes,minVotes,recused
0,1946-001,1946,8,1,0.0
1,1946-001,1946,8,1,0.0
2,1946-001,1946,8,1,0.0
3,1946-001,1946,8,1,0.0
4,1946-001,1946,8,1,0.0


In [6]:
#grouping by case. counting number of recused justices, and number of justices who were on the court.
cases_46_14 = filtered_votes_46_14.groupby(['caseId', 'year', 'majVotes', 'minVotes']).agg({'recused': np.sum, 'caseId': np.size}).add_suffix('_count').reset_index()
cases_46_14.columns = ['caseId', 'year', 'majVotes', 'minVotes', 'num_recused_justices', 'num_justices_on_court']

In [7]:
#creating categories for types of decisions I care about

#even split (4-4 or 3-3)
cases_46_14.loc[cases_46_14['majVotes'] == cases_46_14['minVotes'], 'even_split'] = 1

#unanimous
cases_46_14.loc[(cases_46_14['majVotes'] == 0) | (cases_46_14['minVotes'] == 0), 'unanimous'] = 1

#5-4 split
cases_46_14.loc[((cases_46_14['majVotes'] == 5) & (cases_46_14['minVotes'] == 4)) | ((cases_46_14['majVotes'] == 4) & (cases_46_14['minVotes'] == 5)), 'split_5_4'] = 1

#replace NaNs
cases_46_14 = cases_46_14.fillna(value=0)

#other decision
cases_46_14.loc[(cases_46_14['even_split'] + cases_46_14['unanimous'] + cases_46_14['split_5_4']) == 0 , 'other_decision'] = 1

#replace NaNs
cases_46_14 = cases_46_14.fillna(value=0)

#sanity check. how's my data?
cases_46_14.head()

,caseId,year,majVotes,minVotes,num_recused_justices,num_justices_on_court,even_split,unanimous,split_5_4,other_decision
0,1946-001,1946,8,1,0.0,9,0.0,0.0,0.0,1.0
1,1946-002,1946,6,3,0.0,9,0.0,0.0,0.0,1.0
2,1946-003,1946,5,4,0.0,9,0.0,0.0,1.0,0.0
3,1946-004,1946,5,3,1.0,9,0.0,0.0,0.0,1.0
4,1946-005,1946,6,3,0.0,9,0.0,0.0,0.0,1.0


In [8]:
filtered_cases_46_14 = cases_46_14[['caseId', 'year', 'even_split', 'unanimous', 'split_5_4', 'other_decision', 'num_recused_justices', 'num_justices_on_court',]]

### Oyez API (2015 Cases)
#### Reading in and filtering / grouping the data I need

In [9]:
#Oyez API
#Reading in the JSON
with open('data/2015-cases.json') as data_file:    
    raw_case_list_2015 = json.load(data_file)

In [10]:
#pulling out the URLs of each case for this term
cases_2015_urls = []
for case in raw_case_list_2015:
    cases_2015_urls.append(case['href'])

In [11]:
#Extracting the data I need from each case from this term
filtered_cases = []
for case_url in cases_2015_urls:
    case_dict = {}
    case = urllib.urlopen(case_url)
    case_data = json.loads(case.read())
    case_dict['ID'] = case_data['ID']
    case_dict['name'] = case_data['name']
    if case_data['decisions'] is None:
        case_dict['decided'] = False
    else:
        case_dict['decided'] = True
        case_dict['votes'] = []
        case_dict['majority'] = 0
        case_dict['minority'] = 0
        case_dict['none'] = 0
        case_dict['href'] = case_data['href']
        case_dict['num_votes'] = 0
        for a in case_data['decisions']:
            case_dict['decision_type'] = a['decision_type']
            for b in a['votes']:
                case_dict['num_votes'] += 1
                case_dict['votes'].append(b['vote'])
                if b['vote'] == 'majority':
                    case_dict['majority'] +=1
                elif b['vote'] == 'minority':
                    case_dict['minority'] +=1
                elif b['vote'] == 'none':
                    case_dict['none'] +=1
                else:
                    print "there is another value and it is"
                    print b['vote']
                    print 'and the case is ', case_data['name']
    filtered_cases.append(case_dict)

In [12]:
#Where are we at in the term. Making sure the data is up-to-date
total_cases = len(filtered_cases)
how_many_decided = 0
for a in filtered_cases:
    if a['decided'] is True:
        how_many_decided +=1

print "there have been", str(how_many_decided), "cases decided out of", str(total_cases), "so there are", str((total_cases - how_many_decided)), "to go" 

there have been 72 cases decided out of 79 so there are 7 to go


In [13]:
#getting data from JSON into Pandas
pd_2015_all_cases = pd.io.json.json_normalize(filtered_cases)
pd_2015_all_cases.head()

,ID,decided,decision_type,href,majority,minority,name,none,num_votes,votes
0,56176,True,majority opinion,https://api.oyez.org/cases/2015/14-7505,8.0,1.0,Hurst v. Florida,0.0,9.0,"[majority, majority, majority, majority, major..."
1,56177,True,majority opinion,https://api.oyez.org/cases/2015/14-280,6.0,3.0,Montgomery v. Louisiana,0.0,9.0,"[majority, majority, majority, majority, major..."
2,56178,True,majority opinion,https://api.oyez.org/cases/2015/14-462,6.0,3.0,"DIRECTV, Inc. v. Imburgia",0.0,9.0,"[majority, majority, majority, majority, major..."
3,56180,True,majority opinion,https://api.oyez.org/cases/2015/14-723,8.0,1.0,Montanile v. Board of Trustees of the National...,0.0,9.0,"[majority, majority, majority, majority, major..."
4,56181,True,majority opinion,https://api.oyez.org/cases/2015/14-449,16.0,2.0,Kansas v. Carr,0.0,18.0,"[majority, majority, majority, majority, major..."


In [14]:
#updating cases that are not yet considered decided by Oyez, but were decided morning of 6-23
#fisher 4-3 majority
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62309, 'decided'] = True
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62309, 'decision_type'] = 'majority opinion'
#dollar store 4-4
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62306, 'decided'] = True
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62306, 'decision_type'] = 'equally divided'
#us v texas 4-4
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62654, 'decided'] = True
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62654, 'decision_type'] = 'equally divided'
#birchfield 5-3
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62638, 'decided'] = True
pd_2015_all_cases.loc[pd_2015_all_cases['ID'] == 62638, 'decision_type'] = 'majority opinion'



In [15]:
#Just using cases decided so far this year
decided_2015_cases = pd_2015_all_cases[pd_2015_all_cases['decided'] == True]


In [16]:
#Adding useful columns
decided_2015_cases.loc[decided_2015_cases['decided'] == True, 'year'] = 2015
decided_2015_cases.head()


//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:284: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:461: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,ID,decided,decision_type,href,majority,minority,name,none,num_votes,votes,year
0,56176,True,majority opinion,https://api.oyez.org/cases/2015/14-7505,8.0,1.0,Hurst v. Florida,0.0,9.0,"[majority, majority, majority, majority, major...",2015.0
1,56177,True,majority opinion,https://api.oyez.org/cases/2015/14-280,6.0,3.0,Montgomery v. Louisiana,0.0,9.0,"[majority, majority, majority, majority, major...",2015.0
2,56178,True,majority opinion,https://api.oyez.org/cases/2015/14-462,6.0,3.0,"DIRECTV, Inc. v. Imburgia",0.0,9.0,"[majority, majority, majority, majority, major...",2015.0
3,56180,True,majority opinion,https://api.oyez.org/cases/2015/14-723,8.0,1.0,Montanile v. Board of Trustees of the National...,0.0,9.0,"[majority, majority, majority, majority, major...",2015.0
4,56181,True,majority opinion,https://api.oyez.org/cases/2015/14-449,16.0,2.0,Kansas v. Carr,0.0,18.0,"[majority, majority, majority, majority, major...",2015.0


In [17]:
#recusal--done manually. must update for future cases
decided_2015_cases.loc[decided_2015_cases['ID'] == 62308, 'num_recused_justices'] = 1
decided_2015_cases.loc[decided_2015_cases['ID'] == 62632, 'num_recused_justices'] = 1
decided_2015_cases.loc[decided_2015_cases['ID'] == 62309, 'num_recused_justices'] = 1


#num justices on the court
decided_2015_cases.loc[decided_2015_cases['num_votes'] % 9 == 0, 'num_justices_on_court'] = 9
decided_2015_cases.loc[decided_2015_cases['num_votes'] % 8 == 0, 'num_justices_on_court'] = 8

#evenly split cases
decided_2015_cases.loc[(decided_2015_cases['decision_type'] == 'equally divided'), 'even_split'] = 1

#unanimous
#this is more confusing than the previous dataset, as per curium opinions are considered unanimous but not represented as such
decided_2015_cases.loc[(decided_2015_cases['decision_type'] == 'per curiam') | ((decided_2015_cases['majority'] == 0) & (decided_2015_cases['minority'] == decided_2015_cases['num_votes'])) | ((decided_2015_cases['minority'] == 0) & (decided_2015_cases['majority'] == decided_2015_cases['num_votes'])) | ((decided_2015_cases['majority'] + decided_2015_cases['minority']) == 1), 'unanimous'] = 1

#no 5-4s this year
decided_2015_cases.loc[decided_2015_cases['decided'] == True, 'split_5_4'] = 0


#replace NaNs
decided_2015_cases = decided_2015_cases.fillna(value=0)

#other decision
decided_2015_cases.loc[(decided_2015_cases['even_split'] + decided_2015_cases['unanimous']) == 0 , 'other_decision'] = 1

#replace NaNs
decided_2015_cases = decided_2015_cases.fillna(value=0)






In [18]:
filtered_2015_cases = decided_2015_cases[['ID', 'year', 'even_split', 'unanimous', 'split_5_4', 'other_decision', 'num_recused_justices', 'num_justices_on_court' ]]

In [19]:
filtered_2015_cases.head()

,ID,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court
0,56176,2015.0,0.0,0.0,0.0,1.0,0.0,9.0
1,56177,2015.0,0.0,0.0,0.0,1.0,0.0,9.0
2,56178,2015.0,0.0,0.0,0.0,1.0,0.0,9.0
3,56180,2015.0,0.0,0.0,0.0,1.0,0.0,9.0
4,56181,2015.0,0.0,0.0,0.0,1.0,0.0,9.0


In [20]:
filtered_2015_cases.columns = ['caseId', 'year', 'even_split', 'unanimous', 'split_5_4', 'other_decision', 'num_recused_justices', 'num_justices_on_court']

## Concatentating the two dataframes

In [21]:
filtered_cases_46_14 = filtered_cases_46_14.set_index('caseId')

In [22]:
filtered_2015_cases = filtered_2015_cases.set_index('caseId')

In [23]:
all_cases = pd.concat([filtered_cases_46_14, filtered_2015_cases])

In [24]:
all_cases.head(10)

,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court
caseId,,,,,,,
1946-001,1946.0,0.0,0.0,0.0,1.0,0.0,9.0
1946-002,1946.0,0.0,0.0,0.0,1.0,0.0,9.0
1946-003,1946.0,0.0,0.0,1.0,0.0,0.0,9.0
1946-004,1946.0,0.0,0.0,0.0,1.0,1.0,9.0
1946-005,1946.0,0.0,0.0,0.0,1.0,0.0,9.0
1946-006,1946.0,0.0,0.0,0.0,1.0,1.0,9.0
1946-007,1946.0,0.0,1.0,0.0,0.0,3.0,9.0
1946-008,1946.0,0.0,1.0,0.0,0.0,0.0,9.0
1946-009,1946.0,0.0,1.0,0.0,0.0,0.0,9.0


In [25]:
all_cases.tail(10)

,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court
caseId,,,,,,,
62654,2015.0,1.0,0.0,0.0,0.0,0.0,8.0
62656,2015.0,0.0,0.0,0.0,1.0,0.0,8.0
62657,2015.0,0.0,1.0,0.0,0.0,0.0,8.0
62660,2015.0,0.0,0.0,0.0,1.0,0.0,8.0
62662,2015.0,0.0,1.0,0.0,0.0,0.0,9.0
62664,2015.0,0.0,0.0,0.0,1.0,0.0,8.0
62675,2015.0,0.0,1.0,0.0,0.0,0.0,8.0
62676,2015.0,0.0,1.0,0.0,0.0,0.0,8.0
62691,2015.0,0.0,1.0,0.0,0.0,0.0,8.0


In [26]:
#how many justices heard case
all_cases['num_justices_decided_case'] = all_cases['num_justices_on_court'] - all_cases['num_recused_justices']
all_cases.head()

,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court,num_justices_decided_case
caseId,,,,,,,,
1946-001,1946.0,0.0,0.0,0.0,1.0,0.0,9.0,9.0
1946-002,1946.0,0.0,0.0,0.0,1.0,0.0,9.0,9.0
1946-003,1946.0,0.0,0.0,1.0,0.0,0.0,9.0,9.0
1946-004,1946.0,0.0,0.0,0.0,1.0,1.0,9.0,8.0
1946-005,1946.0,0.0,0.0,0.0,1.0,0.0,9.0,9.0


# Data for Charts

**CHART ONE: Between 1946 and 2015, how many justices heard each case?**

In [27]:
num_justices_heard_cases = all_cases.groupby('num_justices_decided_case').agg({'num_justices_decided_case': np.size}).add_suffix('_count').reset_index()
num_justices_heard_cases = num_justices_heard_cases.set_index('num_justices_decided_case')
num_justices_heard_cases

,num_justices_decided_case_count
num_justices_decided_case,
5.0,1.0
6.0,30.0
7.0,326.0
8.0,1602.0
9.0,6775.0


In [28]:
num_justices_heard_cases.to_csv('data/chart_1_num_justices_decided_case.tsv', sep='\t')

**CHART TWO: Between 1946 and 2015, what percentage of cases were heard by eight justices, and why?**

In [29]:
#number of cases each year to use as denominator for percentage
cases_per_year = all_cases.groupby(['year']).agg({'year': np.size}).add_suffix('_count').reset_index()
cases_per_year.year = cases_per_year.year.astype(int)
cases_per_year.year_count = cases_per_year.year_count.astype(int)
cases_per_year.head()

,year,year_count
0,1946,142
1,1947,118
2,1948,127
3,1949,99
4,1950,103


In [30]:
eight_justices_vacancy = all_cases[(all_cases['num_justices_decided_case'] == 8) & (all_cases['num_recused_justices'] == 0)]
eight_vacancy_by_year = eight_justices_vacancy.groupby(['year']).agg({'year': np.size}).add_suffix('_count').reset_index()
eight_vacancy_by_year.columns = ['year', 'cases_with_vacancy']
eight_vacancy_by_year.head()

,year,cases_with_vacancy
0,1951.0,1.0
1,1954.0,50.0
2,1956.0,8.0
3,1961.0,4.0
4,1964.0,1.0


In [31]:
eight_justices_recusal = all_cases[(all_cases['num_justices_decided_case'] == 8) & (all_cases['num_recused_justices'] == 1)]
eight_recusal_by_year = eight_justices_recusal.groupby(['year']).agg({'year': np.size}).add_suffix('_count').reset_index()
eight_recusal_by_year.columns = ['year', 'cases_with_recusal']
eight_recusal_by_year.head()

,year,cases_with_recusal
0,1946.0,13.0
1,1947.0,11.0
2,1948.0,10.0
3,1949.0,61.0
4,1950.0,26.0


In [32]:
#joining three above tables together
years_vacancies = pd.merge(cases_per_year, eight_vacancy_by_year, on='year', how='left')
years_vacancies_recusals = pd.merge(years_vacancies, eight_recusal_by_year, on='year', how='left')
years_vacancies_recusals = years_vacancies_recusals.fillna(value=0)
years_vacancies_recusals.head()

,year,year_count,cases_with_vacancy,cases_with_recusal
0,1946,142,0.0,13.0
1,1947,118,0.0,11.0
2,1948,127,0.0,10.0
3,1949,99,0.0,61.0
4,1950,103,0.0,26.0


In [33]:
years_vacancies_recusals['vacancy_perc'] = years_vacancies_recusals['cases_with_vacancy'] / years_vacancies_recusals['year_count']
years_vacancies_recusals['recusal_perc'] = years_vacancies_recusals['cases_with_recusal'] / years_vacancies_recusals['year_count']
years_vacancies_recusals.head()

,year,year_count,cases_with_vacancy,cases_with_recusal,vacancy_perc,recusal_perc
0,1946,142,0.0,13.0,0.0,0.091549
1,1947,118,0.0,11.0,0.0,0.093220
2,1948,127,0.0,10.0,0.0,0.078740
3,1949,99,0.0,61.0,0.0,0.616162
4,1950,103,0.0,26.0,0.0,0.252427


In [34]:
years_percentages = years_vacancies_recusals[['year', 'vacancy_perc', 'recusal_perc']]
years_percentages['Justice vacancy'] = years_percentages['vacancy_perc'] * 100
years_percentages['Recusal'] = years_percentages['recusal_perc'] * 100


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [35]:
years_percentages_100 = years_percentages[['year', 'Justice vacancy', 'Recusal']]
years_percentages_100.set_index('year', inplace = True)

years_percentages_100.head()

,Justice vacancy,Recusal
year,,
1946,0.0,9.154930
1947,0.0,9.322034
1948,0.0,7.874016
1949,0.0,61.616162
1950,0.0,25.242718


In [36]:
years_percentages_100.to_csv('data/chart_2_eight_justices_by_year.csv')

**CHART THREE: Types of decisions in cases heard by eight justices between 1946 and 2014**

In [37]:
#Cases heard by eight justices excluding 2015
eight_justices = all_cases[(all_cases['num_justices_decided_case'] == 8) & (all_cases['year'] != 2015)]
eight_justices.head()

,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court,num_justices_decided_case
caseId,,,,,,,,
1946-004,1946.0,0.0,0.0,0.0,1.0,1.0,9.0,8.0
1946-006,1946.0,0.0,0.0,0.0,1.0,1.0,9.0,8.0
1946-010,1946.0,0.0,1.0,0.0,0.0,1.0,9.0,8.0
1946-015,1946.0,0.0,1.0,0.0,0.0,1.0,9.0,8.0
1946-062,1946.0,0.0,0.0,0.0,1.0,1.0,9.0,8.0


In [38]:
eight_justices_filtered = eight_justices[['even_split', 'unanimous', 'split_5_4', 'other_decision']]

In [39]:
eight_justices_type_count = eight_justices_filtered.sum()
eight_justices_type_count

even_split        110.0
unanimous         639.0
split_5_4           0.0
other_decision    798.0
dtype: float64

In [40]:
#creating dataframe and getting in place to export to csv
eight_justices_type_count_pd = pd.DataFrame(eight_justices_type_count, columns=['count'])
eight_justices_type_count_pd.reset_index(inplace = True)
eight_justices_type_count_pd.columns = ['type','count']
eight_justices_type_count_pd.set_index('type', inplace = True)
eight_justices_type_count_pd

,count
type,
even_split,110.0
unanimous,639.0
split_5_4,0.0
other_decision,798.0


In [41]:
eight_justices_type_count_pd.to_csv('data/chart_3_eight_justices_type_count.tsv', sep='\t')

**CHART FOUR: Types of decisions in cases heard by eight justices in 2015**

In [42]:
#Cases heard by eight justices in 2015
eight_justices_2015 = all_cases[(all_cases['num_justices_decided_case'] == 8) & (all_cases['year'] == 2015)]
eight_justices_2015.head()

,year,even_split,unanimous,split_5_4,other_decision,num_recused_justices,num_justices_on_court,num_justices_decided_case
caseId,,,,,,,,
62164,2015.0,0.0,0.0,0.0,1.0,0.0,8.0,8.0
62165,2015.0,0.0,0.0,0.0,1.0,0.0,8.0,8.0
62168,2015.0,0.0,1.0,0.0,0.0,0.0,8.0,8.0
62301,2015.0,0.0,0.0,0.0,1.0,0.0,8.0,8.0
62306,2015.0,1.0,0.0,0.0,0.0,0.0,8.0,8.0


In [43]:
eight_justices_2015_filtered = eight_justices_2015[['even_split', 'unanimous', 'split_5_4', 'other_decision']]

In [44]:
eight_justices_2015_type_count = eight_justices_2015_filtered.sum()

In [45]:
#creating dataframe and getting in place to export to csv
eight_justices_2015_type_count_pd = pd.DataFrame(eight_justices_2015_type_count, columns=['count'])
eight_justices_2015_type_count_pd.reset_index(inplace = True)
eight_justices_2015_type_count_pd.columns = ['type','count']
eight_justices_2015_type_count_pd.set_index('type', inplace = True)
eight_justices_2015_type_count_pd

,count
type,
even_split,4.0
unanimous,27.0
split_5_4,0.0
other_decision,24.0


In [46]:
eight_justices_2015_type_count_pd.to_csv('data/chart_4_eight_justices_2015_type_count.tsv', sep='\t')

**CHART FIVE: Types of decisions in all cases**

In [47]:
all_cases_filtered = all_cases[['even_split', 'unanimous', 'split_5_4', 'other_decision']]
all_cases_type_count = all_cases_filtered.sum()


In [48]:
#creating dataframe and getting in place to export to csv


all_cases_type_count_pd = pd.DataFrame(all_cases_type_count, columns=['count'])
all_cases_type_count_pd.reset_index(inplace = True)
all_cases_type_count_pd.columns = ['type', 'count']
all_cases_type_count_pd.set_index('type', inplace=True)
all_cases_type_count_pd


,count
type,
even_split,117.0
unanimous,3453.0
split_5_4,1295.0
other_decision,3869.0


In [49]:
all_cases_type_count_pd.to_csv('data/chart_5_all_cases_type_count.tsv', sep='\t')

**For stat in article, of eight person cases, what is the reason?

In [50]:
eight_justices.groupby('num_recused_justices').agg({'num_recused_justices': np.size})

,num_recused_justices
num_recused_justices,
0.0,242.0
1.0,1305.0


In [51]:
1305.0 / (242 + 1305)

0.843568196509373